In [1]:
import pandas as _pd
import plotly.express as _px

In [2]:
dataset_path = 'data/Salary.csv'

In [3]:
def read_file(path:str):
    tbl = _pd.read_csv(dataset_path, sep=',', encoding='utf-8')
    
    return tbl 

In [4]:
tbl = read_file(dataset_path)
tbl

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,32.0,Male,1,Software Engineer,5.0,90000.0,UK,White,0
1,28.0,Female,2,Data Analyst,3.0,65000.0,USA,Hispanic,0
2,45.0,Male,3,Manager,15.0,150000.0,Canada,White,1
3,36.0,Female,1,Sales Associate,7.0,60000.0,USA,Hispanic,0
4,52.0,Male,2,Director,20.0,200000.0,USA,Asian,0
...,...,...,...,...,...,...,...,...,...
6679,49.0,Female,3,Director of Marketing,20.0,200000.0,UK,Mixed,0
6680,32.0,Male,0,Sales Associate,3.0,50000.0,Australia,Australian,0
6681,30.0,Female,1,Financial Manager,4.0,55000.0,China,Chinese,0
6682,46.0,Male,2,Marketing Manager,14.0,140000.0,China,Korean,0


In [117]:
ds1 = tbl.groupby(['Job Title']).agg({'Salary':'mean'}).sort_values(by='Salary',ascending=True)
ds1

fig = _px.bar(ds1, x=ds1.index, y=['Salary'])
fig.show()

In [6]:
tbl.groupby(['Country'])['Salary'].mean().sort_values(ascending=False)

Country
Canada       116455.090772
China        116282.589993
UK           115919.916667
Australia    114925.465169
USA          112998.758112
Name: Salary, dtype: float64

In [7]:
tbl.groupby(['Gender'])['Salary'].mean().sort_values(ascending=False)

Gender
Male      121395.697630
Female    107888.998672
Name: Salary, dtype: float64

In [8]:
tbl.groupby(['Job Title', 'Salary'])['Years of Experience'].mean().sort_values(ascending=False)

Job Title                  Salary  
Software Engineer Manager  188651.0    34.0
                           179180.0    33.0
Full Stack Engineer        188232.0    33.0
Software Engineer Manager  191790.0    33.0
                           191267.0    32.0
                                       ... 
Project Engineer           52831.0      0.0
Receptionist               25000.0      0.0
Project Engineer           51831.0      0.0
Back end Developer         55538.0      0.0
                           51832.0      0.0
Name: Years of Experience, Length: 1040, dtype: float64

In [9]:
tbl.groupby(['Country','Job Title'])['Salary'].mean().sort_values(ascending=False)

Country  Job Title                  
Canada   CEO                            250000.000000
         Chief Technology Officer       250000.000000
USA      Chief Data Officer             220000.000000
Canada   Director of Data Science       207692.307692
UK       Director of Data Science       205555.555556
                                            ...      
USA      Receptionist                    25000.000000
China    Receptionist                    25000.000000
Canada   Receptionist                    25000.000000
USA      HR Coordinator                  23833.333333
         Business Operations Analyst       350.000000
Name: Salary, Length: 354, dtype: float64

In [10]:
_pd.options.plotting.backend = "plotly"

In [11]:
view_1 = tbl.groupby(['Country'])['Salary'].mean()

fig2 = _px.bar(
    view_1, 
    orientation='h',
    text_auto=True
)
fig2.show()

In [12]:
view_1 = tbl.groupby(['Job Title'])['Salary'].mean().sort_values(ascending=True)

fig2 = _px.bar(
    view_1, 
    orientation='h',
    text_auto=True,
    range_x=[-0.5, 1.5],
    barmode = 'overlay'
)
fig2.show()